# Chapter 6, Functions

This is, in theory, the chapter where they will explain how common is to use functions inside functions with functions... It should be fun even when we had been doing this from the beginning.

### Functions applying functions

Let's get the double of a list

In [1]:
let rec double l =
  match l with
  | hd :: tl -> (hd * 2) :: double tl
  | _ -> [] ;;

double [1;2;3;4]

val double : int list -> int list = <fun>


- : int list = [2; 4; 6; 8]


We can simplify with a function to apply functions, let's call it `map`, notice the signature!

In [2]:
(* This function is not tail recursive, 
 * I know I can do it better *)
let rec map f l =
  match l with
  | hd :: tl -> (f hd) :: map f tl
  | _ -> [] ;;

val map : ('a -> 'b) -> 'a list -> 'b list = <fun>


When we see a signature like that `('a -> 'b)` it basically means "_a function taking `'a` and returning `'b`_". Notice how OCaml is smart enough to infer the correct type for the parameters, you don't have to tell a damn and it is still strong typed.

When we are using the function just once and we don't want to name it, we can do it as well:

In [3]:
map (fun x -> x * 2) [1;2;3;4]

- : int list = [2; 4; 6; 8]


We can change our `merge` and `msort` functions to use our own operators for comparisson, in this case the operators should have the signature `('a -> 'a -> bool)` so the `merge` would be defined as `merge: ('a -> 'a -> bool) -> 'a list -> 'a list -> 'a list` and `msort` as `msort: ('a -> 'a -> bool) -> 'a list -> 'a list`:

## Exercises

 * Write a simple recursive function calm to replace exclamation marks in a char list with periods. For example `calm ['H'; 'e'; 'l'; 'p'; '!'; ' '; 'F'; 'i'; 'r'; 'e'; '!']` should evaluate to `calm ['H'; 'e'; 'l'; 'p'; '.'; ' '; 'F'; 'i'; 'r'; 'e'; '.']`. Now rewrite your function to use map instead of recursion. What are the types of your functions?

In [5]:
let rec calm l =
  match l with
  | hd :: tl -> if hd = '!' then '.' :: calm tl else hd :: calm tl
  | _ -> [] ;;
  
calm ['H'; 'e'; 'l'; 'p'; '!'; ' '; 'F'; 'i'; 'r'; 'e'; '!']

val calm : char list -> char list = <fun>


- : char list = ['H'; 'e'; 'l'; 'p'; '.'; ' '; 'F'; 'i'; 'r'; 'e'; '.']


In [7]:
let calm' l =
  map (fun x -> if x = '!' then '.' else x) l ;;

calm' ['H'; 'e'; 'l'; 'p'; '!'; ' '; 'F'; 'i'; 'r'; 'e'; '!']

val calm' : char list -> char list = <fun>


- : char list = ['H'; 'e'; 'l'; 'p'; '.'; ' '; 'F'; 'i'; 'r'; 'e'; '.']


 * Write a function `clip` which, given an integer, clips it to the range 1…10 so that integers bigger than 10 round down to 10, and those smaller than 1 round up to 1. Write another function `cliplist` which uses this first function together with `map` to apply this clipping to a whole list of integers.

In [20]:
let clip n =
  if n < 1 then 1 else
  if n > 10 then 10 else n ;;
  
let cliplist l =
  map clip l ;;

cliplist [1; 14; 2; 5; (-15); 0]

val clip : int -> int = <fun>


val cliplist : int list -> int list = <fun>


- : int list = [1; 10; 2; 5; 1; 1]


 * Express your function `cliplist` again, this time using an anonymous function instead of `clip`.

In [16]:
let cliplist' l =
  map (fun x -> if x < 1 then 1 else if x > 10 then 10 else x) l ;;
  
cliplist' [1; 14; 2; 5; (-15); 0]

val cliplist' : int list -> int list = <fun>


- : int list = [1; 10; 2; 5; 1; 1]


 * Write a function `apply` which, given another function, a number of times to apply it, and an initial argument for the function, will return the cumulative effect of repeatedly applying the function. For instance, `apply f 6 4` should return `f (f (f (f (f (f 4))))))`. What is the type of your function?

_I guess for this the type should be `('a -> 'b) -> int -> 'a -> 'b`_

In [19]:
let rec apply f n x =
  if n = 0 then x else
  apply f (n - 1) (f x) ;;

val apply : ('a -> 'a) -> int -> 'a -> 'a = <fun>


_I was pleasenly surprised I was wrong with guessing the type! if you think about I would say the type of the function was `('a -> 'b)` but it doesn't make sense (and OCaml knew it) because you apply the function again with the same type as the entry parameter, so it should be `('a -> 'a)` and the type of the whole function should be then `('a -> 'a) -> int -> 'a -> 'a`_

* Modify the insertion sort function from the preceding chapter to take a comparison function, in the same way that we modified merge sort in this chapter. What is its type?

_I don't want to really do it, but I can guess its type, it would be `sort: ('a -> 'a -> bool) -> 'a list -> 'a list`._

 * Write a function `filter` which takes a function of type `α → bool` and an `α list` and returns a list of just those elements of the argument list for which the given function returns true.

In [26]:
(* Guess what? tail recursive function again yo! *)

let filter f l =
  let rec loop f l' acc =
    match l' with
    | hd :: tl -> 
      let acc' = if f hd then hd :: acc else acc in
      loop f tl acc'
    | _ -> acc in
  loop f l [] ;;
  
filter (fun x -> x = true) [true; true; false; true]

val filter : ('a -> bool) -> 'a list -> 'a list = <fun>


- : bool list = [true; true; true]


 * Write the function `for_all` which, given a function of type `α → bool` and an argument list of type `α list` evaluates to true if and only if the function returns true for every element of the list. Give examples of its use.

In [31]:
(* And this is tail recursive, 
 * after doing it a few times it becomes a normal pattern :D *)
 
let for_all f l =
  let rec loop l' x =
    if x = false then x else
    match l' with
    | hd :: tl -> loop tl (f hd)
    | _ -> x in
  loop l true ;;

val for_all : ('a -> bool) -> 'a list -> bool = <fun>


 * Write a function `mapl` which maps a function of type `α → β` over a list of type `α list list` to produce a list of type `β list list`.

In [42]:
(* We will need a reverse function here, for the sake of simplicity 
 * we will use the one in the standard library *)

(* Let's make a better map function *)
let map' f l =
  let rec loop l' acc =
    match l' with
    | hd :: tl ->
      let x = f hd in
      loop tl (x :: acc)
    | _ -> acc in
  loop (List.rev l) [] ;;

let mapl f l =
  let rec loop l' acc =
    match l' with
    | hd :: tl ->
      let x = map' f hd in
      loop tl (x :: acc)
    | _ -> acc in
  loop (List.rev l) [] ;;
  
mapl (fun x -> x * 2) [[2;4;8];[3;6;9]]

val map' : ('a -> 'b) -> 'a list -> 'b list = <fun>


val mapl : ('a -> 'b) -> 'a list list -> 'b list list = <fun>


- : int list list = [[4; 8; 16]; [6; 12; 18]]
